In [ ]:
Source: The Webpage has a compiled ratings of over 1700 Chocolate bars (table format)
Ratings are from 1-5 => 1 is “unpleasant” chocolate, while a 5 is a bar that transcends “beyond the ordinary limits”

#Questions
Where are the best cocoa beans grown? 
Which countries produce the highest-rated bars? 
What’s the relationship between cocoa solids percentage and rating?
Where are the best cocoa beans grown..best bar names?
Which countries produce the highest-rated bars

In [ ]:

from bs4 import BeautifulSoup
import requests
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# inspect table! what data/columns are needed?
# make a request for the raw HTML data
webpage = requests.get("https://s3.amazonaws.com/codecademy-content/courses/beautifulsoup/cacao/index.html")

# create a BS object to tranverse the HTML
soup = BeautifulSoup(webpage.content, "html.parser")
# Explore the soup obj.
# print(soup)

"""
How are ratings distributed?
observe the dataset distribution via histogram

"""
rating_tags = soup.find_all(attrs={"class":"Rating"})

#print(td_tag_ratings)   #<td class="Rating">Rating</td>

#store all the ratings 
ratings = []

#Loop through the ratings_tags and get the text contained in each one. Add it to the ratings list.
for td in rating_tags[1:]:
    floated_rating = float(td.get_text())
    ratings.append(floated_rating)
#print(ratings)

"""
td_tag_ratings[1:] avoids the first elament which contains the header string "Ratings"
.get_text() gets the innerHTML of the td tag
float() converts ratings to numericals
"""

#plt.hist(ratings)
#plt.show()

"""
Which chocolatier makes the best chocolate? groupby (highest average rating)
10 most highly rated chocolatiers

"""
company_tags = soup.select(".Company")
companies = []

#loop through company_tags and add to companies list
for td in company_tags[1:]:
    companies.append(td.get_text())
#print(companies)

#create Dataframe from the Ratings and companies lists
cocoa_dict = {"Company":companies, "Rating":ratings}
cocoa_df = pd.DataFrame.from_dict(cocoa_dict)
#print(cocoa_df)

#calculate mean ratings and the 10 best rated companies
mean_ratings = cocoa_df.groupby("Company").Rating.mean()
best_10 =  mean_ratings.nlargest(10)
#print(mean_ratings)
#print(best_10)

"""
Is more cacao better? higher rating?
"""

cocoa_percent_tags = soup.select(".CocoaPercent")
cocoa_percents = []

for td in cocoa_percent_tags[1:]:
    percent = float(td.get_text().strip("%"))
    cocoa_percents.append(percent)
#print(cocoa_percents)

#add a new column to cocoa_df
cocoa_df["CocoaPercentage"] = cocoa_percents
print(cocoa_df)

#make a scatter plot to observe effect of cocoa % on rating
#Call plt.clf() to clear the figure between showing the histogram and this scatterplot.
plt.clf()

plt.scatter(cocoa_df.CocoaPercentage, cocoa_df.Rating)
plt.xlabel("Cocoa Percentage")
plt.ylabel("Chocolate Rating")
#plt.show()

"""Is there any correlation here?"""
#draw a line of best-fit over the scatterplot.
z = np.polyfit(cocoa_df.CocoaPercentage, cocoa_df.Rating, 1)

line_function = np.poly1d(z)
plt.plot(cocoa_df.CocoaPercentage, line_function(cocoa_df.CocoaPercentage), "r--")

plt.show()

"""Where are the best cocoa beans grown..best bar names? """
specific_bean_origin_tags = soup.select(".Origin")
#print(specific_bean_origin_tags)
specific_origin =[]

for td in specific_bean_origin_tags[1:]:
    specific_origin.append(td.get_text())
#print(specific_origin)

#add a new column to cocoa_df
cocoa_df["SpecificBeanOrigin"] = specific_origin

#ratings vs specific origin
mean_ratings_1 = cocoa_df.groupby("SpecificBeanOrigin").Rating.mean()
best_15_bar_name =  mean_ratings_1.nlargest(15)
#print(best_15_bar_name)

"""Which countries produce the highest-rated bars?"""
broad_bean_origin_tags = soup.select(".BroadBeanOrigin")
countries = []
for td in broad_bean_origin_tags[1:]:
    countries.append(td.get_text())
#print(countries)

#add a new column to cocoa_df
cocoa_df["Country"] = countries

#ratings vs country
mean_ratings_2 = cocoa_df.groupby("Country").Rating.mean()
best_10_bar_name =  mean_ratings_2.nlargest(10)

#print(best_10_bar_name)

#print(cocoa_df)



